In [20]:
#import dataset
import kagglehub
path = kagglehub.dataset_download("awsaf49/brats2020-training-data")
print ("Path to dataset files:", path)


100%|██████████| 6.76G/6.76G [02:35<00:00, 46.7MB/s]


Extracting files...
Path to dataset files: /Users/stemesghen/.cache/kagglehub/datasets/awsaf49/brats2020-training-data/versions/3


In [21]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv3D, Conv3DTranspose, MaxPooling3D, concatenate, UpSampling3D



In [10]:
def unet(input_size=(128, 128, 128, 1)):  # Input size: (depth, height, width, channels)
    inputs = Input(input_size)  # Tensor shape: (128, 128, 128, 1)

    # *** First DownConvolution / Encoder Leg ***
    conv1 = Conv3D(filters=64, kernel_size=(3, 3, 3), padding="same")(inputs)  # Tensor shape: (128, 128, 128, 64)
    bn1 = Activation("relu")(conv1)
    conv1 = Conv3D(filters=64, kernel_size=(3, 3, 3), padding="same")(bn1)  # Tensor shape: (128, 128, 128, 64)
    bn1 = BatchNormalization(axis=-1)(conv1)
    bn1 = Activation("relu")(bn1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(bn1)  # Tensor shape after pooling: (64, 64, 64, 64)

    # *** Second DownConvolution / Encoder Leg ***
    conv2 = Conv3D(filters=128, kernel_size=(3, 3, 3), padding="same")(pool1)  # Tensor shape: (64, 64, 64, 128)
    bn2 = Activation("relu")(conv2)
    conv2 = Conv3D(filters=128, kernel_size=(3, 3, 3), padding="same")(bn2)  # Tensor shape: (64, 64, 64, 128)
    bn2 = BatchNormalization(axis=-1)(conv2)
    bn2 = Activation("relu")(bn2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(bn2)  # Tensor shape after pooling: (32, 32, 32, 128)

    # *** Third DownConvolution / Encoder Leg ***
    conv3 = Conv3D(filters=256, kernel_size=(3, 3, 3), padding="same")(pool2)  # Tensor shape: (32, 32, 32, 256)
    bn3 = Activation("relu")(conv3)
    conv3 = Conv3D(filters=256, kernel_size=(3, 3, 3), padding="same")(bn3)  # Tensor shape: (32, 32, 32, 256)
    bn3 = BatchNormalization(axis=-1)(conv3)
    bn3 = Activation("relu")(bn3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(bn3)  # Tensor shape after pooling: (16, 16, 16, 256)

    # *** Fourth DownConvolution / Encoder Leg ***
    conv4 = Conv3D(filters=512, kernel_size=(3, 3, 3), padding="same")(pool3)  # Tensor shape: (16, 16, 16, 512)
    bn4 = Activation("relu")(conv4)
    conv4 = Conv3D(filters=512, kernel_size=(3, 3, 3), padding="same")(bn4)  # Tensor shape: (16, 16, 16, 512)
    bn4 = BatchNormalization(axis=-1)(conv4)
    bn4 = Activation("relu")(bn4)
    pool4 = MaxPooling3D(pool_size=(2, 2, 2))(bn4)  # Tensor shape after pooling: (8, 8, 8, 512)

    # *** Bottleneck Layer ***
    conv5 = Conv3D(filters=1024, kernel_size=(3, 3, 3), padding="same")(pool4)  # Tensor shape: (8, 8, 8, 1024)
    bn5 = Activation("relu")(conv5)
    conv5 = Conv3D(filters=1024, kernel_size=(3, 3, 3), padding="same")(bn5)  # Tensor shape: (8, 8, 8, 1024)
    bn5 = BatchNormalization(axis=-1)(conv5)
    bn5 = Activation("relu")(bn5)

    # *** Decoder / Expanding Path (UpConvolution) ***
    up6 = concatenate([Conv3DTranspose(512, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(bn5), conv4], axis=-1)
    conv6 = Conv3D(filters=512, kernel_size=(3, 3, 3), padding="same")(up6)
    bn6 = Activation("relu")(conv6)
    conv6 = Conv3D(filters=512, kernel_size=(3, 3, 3), padding="same")(bn6)
    bn6 = BatchNormalization(axis=-1)(conv6)
    bn6 = Activation("relu")(bn6)

    up7 = concatenate([Conv3DTranspose(256, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(bn6), conv3], axis=-1)
    conv7 = Conv3D(filters=256, kernel_size=(3, 3, 3), padding="same")(up7)
    bn7 = Activation("relu")(conv7)
    conv7 = Conv3D(filters=256, kernel_size=(3, 3, 3), padding="same")(bn7)
    bn7 = BatchNormalization(axis=-1)(conv7)
    bn7 = Activation("relu")(bn7)

    up8 = concatenate([Conv3DTranspose(128, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(bn7), conv2], axis=-1)
    conv8 = Conv3D(filters=128, kernel_size=(3, 3, 3), padding="same")(up8)
    bn8 = Activation("relu")(conv8)
    conv8 = Conv3D(filters=128, kernel_size=(3, 3, 3), padding="same")(bn8)
    bn8 = BatchNormalization(axis=-1)(conv8)
    bn8 = Activation("relu")(bn8)

    up9 = concatenate([Conv3DTranspose(64, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(bn8), conv1], axis=-1)
    conv9 = Conv3D(filters=64, kernel_size=(3, 3, 3), padding="same")(up9)
    bn9 = Activation("relu")(conv9)
    conv9 = Conv3D(filters=64, kernel_size=(3, 3, 3), padding="same")(bn9)
    bn9 = BatchNormalization(axis=-1)(conv9)
    bn9 = Activation("relu")(bn9)

    # Final output layer for segmentation
    conv10 = Conv3D(filters=1, kernel_size=(1, 1, 1), activation="sigmoid")(bn9)  # Tensor shape: (128, 128, 128, 1)

    return Model(inputs=[inputs], outputs=[conv10])


In [11]:
# function to create dice coefficient
def dice_coef(y_true, y_pred, smooth=100):
    y_true_flatten = K.flatten(y_true)  # Flatten the ground truth mask to a 1D array
    y_pred_flatten = K.flatten(y_pred)  # Flatten the predicted mask to a 1D array

    intersection = K.sum(y_true_flatten * y_pred_flatten)  # Calculate the overlap (intersection) between true and predicted masks
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)  # Sum of all true and predicted pixels (union)
    return (2 * intersection + smooth) / (union + smooth)  # Dice formula with smoothing factor


# function to create dice loss
def dice_loss(y_true, y_pred, smooth=100):
    return -dice_coef(y_true, y_pred, smooth)

# function to create iou coefficient
def iou_coef(y_true, y_pred, smooth=100):
    intersection = K.sum(y_true * y_pred)  # Calculate intersection (overlap between true and predicted masks)
    sum = K.sum(y_true + y_pred)  # Calculate the total sum of true and predicted pixels
    iou = (intersection + smooth) / (sum - intersection + smooth)  # IoU formula with smoothing factor
    return iou


In [14]:
model = unet()
model.compile(Adamax(learning_rate= 0.001), loss= dice_loss, metrics= ['accuracy', iou_coef, dice_coef])

model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 128, 1)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 128, 128,  │      1,792 │ input_layer_1[0]… │
│                     │ 128, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 128, 128,  │          0 │ conv3d_3[0][0]    │
│ (Activation)        │ 128, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_4 (Conv3D)   │ (None, 128, 128,  │    110,656 │ activation[0][0]  │
│                     │ 128, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 128,  │        256 │ conv3d_4[0][0]    │
│ (BatchNormalizatio… │ 128, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_2     │ (None, 64, 64,    │          0 │ activation_1[0][… │
│ (MaxPooling3D)      │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_5 (Conv3D)   │ (None, 64, 64,    │    221,312 │ max_pooling3d_2[… │
│                     │ 64, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 64, 64,    │          0 │ conv3d_5[0][0]    │
│ (Activation)        │ 64, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_6 (Conv3D)   │ (None, 64, 64,    │    442,496 │ activation_2[0][… │
│                     │ 64, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ conv3d_6[0][0]    │
│ (BatchNormalizatio… │ 64, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_3     │ (None, 32, 32,    │          0 │ activation_3[0][… │
│ (MaxPooling3D)      │ 32, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_7 (Conv3D)   │ (None, 32, 32,    │    884,992 │ max_pooling3d_3[… │
│                     │ 32, 256)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 32, 32,    │          0 │ conv3d_7[0][0]    │
│ (Activation)        │ 32, 256)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_8 (Conv3D)   │ (None, 32, 32,    │  1,769,728 │ activation_4[0][… │
│                     │ 32, 256)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │      1,024 │ conv3d_8[0][0]  

 Total params: 90,304,449 (344.48 MB)

 Trainable params: 90,298,561 (344.46 MB)

 Non-trainable params: 5,888 (23.00 KB)

In [15]:
model.compile(
    optimizer=Adamax(learning_rate=0.001),  # Adamax optimizer with a learning rate of 0.001
    loss=dice_loss,                         # Dice loss for segmentation
    metrics=['accuracy', iou_coef, dice_coef]  # Metrics to track
)

In [13]:
#Contraction Path
#DownConv1 (3,3,3)
#maxpool (2,2,2)
#Down Conv3
#maxpool 
#down conv
#max pool
#down conv 4



In [ ]:
#Expanding Path

#upsampling
#up convolutions
#up sample
#upconv
#upsample
#upconv


In [ ]:
# out put for probability of a pixel in the image